In [ ]:
import logging
import shutil
import sys
import traceback
from pathlib import Path

import toml

sys.path.append(str(Path("..").resolve()))
from src.pixseg.pipeline import Config, init_logging
from src.pixseg.utils.rng import seed

**Declare path to config**

Either a config file or a folder of config files, which will be ran through one by one

In [ ]:
CONFIG_PATH = Path("config.toml")

In [ ]:
config_files: list[Path] = []
if CONFIG_PATH.is_file():
    config_files = [CONFIG_PATH]
else:
    config_files = list(CONFIG_PATH.glob("*.*"))

In [ ]:
for config_file in config_files:
    config_dict = toml.load(config_file)

    if len(config_files) > 1:
        # use folder name as wandb group and file stem as notes
        wandb_params: dict = config_dict["log"]["wandb"]["params"]
        wandb_params.update({"group": CONFIG_PATH.name, "notes": config_file.stem})

    config = Config(config_dict)
    out_folder = config.out_folder
    print(f"Experiment {config_file} outputs to {out_folder}")

    out_folder.mkdir(parents=True)
    shutil.copy(config_file, out_folder / config_file.name)
    with init_logging(out_folder / "run.log"):
        config_to_log = {k: v for k, v in config_dict.items() if k != "log"}
        config_text = toml.dumps(config_to_log)
        logging.debug(f"Config:\n{config_text}")

        try:
            trainer = config.to_trainer()
            seed(42)
            trainer.train()
        except BaseException as e:
            print("Error occurred. Terminating this training ...")
            traceback.print_exc()